In [1]:
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf

In [2]:
model = tf.keras.models.load_model('isl_neural_network.h5')


In [3]:
num_list = [str(num) for num in range(1, 10)]
letter_list = [chr(letter) for letter in range(ord('A'), ord('Z') + 1)]
action_labels = num_list + letter_list


In [4]:
single = [0]*63


In [5]:
mp_hands = mp.solutions.hands.Hands(static_image_mode=False, max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils


In [16]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = mp_hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        hand_landmarks = results.multi_hand_landmarks[0]

        landmarks = []
        for landmark in hand_landmarks.landmark:
            landmarks.append(landmark.x)
            landmarks.append(landmark.y)
            landmarks.append(landmark.z)
#         
        
        if len(landmarks)==63:
            landmarks = landmarks + single

#             landmarks = np.concatenate((landmarks,single),axis=1)
        landmarks = np.array(landmarks)
        landmarks = np.reshape(landmarks, (1, 126, 1))

        # Perform action classification using the LSTM model
        predictions = model.predict(landmarks)
        predicted_action = action_labels[np.argmax(predictions)]
        
        # Draw landmarks and action label on the frame
#         mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
#                                   mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
#                                   mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2))
        cv2.putText(frame, predicted_action, (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 2.5, (0, 255, 0), 2, cv2.LINE_AA)

    # Display the frame
    cv2.imshow("Hand Action Recognition", frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and destroy windows
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 40ms/step
